# Projeto 1 - Ciência dos Dados

Nome: Diego Saragoza da Sila

Nome: Gabriel Ribeiro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from cleantext import clean_words, clean

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\user\OneDrive - Insper - Institudo de Ensino e Pesquisa\S2\CDADOS\CDADOS-P1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
dados = pd.read_excel('Election.xlsx', 'Treinamento')
dados1 = pd.read_excel('Election.xlsx', 'Teste')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso "produto" é as eleições de 2020 dos Estados Unidos da América. Durante nossas classificações, levamos em conta os nomes dos candidatos aos cargos e o contexto da frase do tweet em questão, caso houvesse alguma evidencia sobre votos ou eleições classificavamos este como relevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [49]:
#https://machinelearningmastery.com/clean-text-machine-learning-python/
#Tutorial para limpeza de textos para machine learning onde retiramos as duas linhas de código abaixo e seu funcionamento.
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def replaceWords(string):
    string = string.lower()
    for word in stop_words:
        string = string.replace(' ' + word.lower() + ' ', ' ')
    string = string.replace('rt', '')
    return string

#Pegando cada frase dos tweets, aplicando a função clean e transformando os dados em uma Series
tweets = []
for phrase in dados['Treinamento']:
    p = replaceWords(phrase)
    p = clean(p, punct = True, extra_spaces = True)
    tweets.append(p)
            
tweets = pd.Series(tweets)
print(tweets)

0      biden aims big michigan win sanders looks keep...
1      wowits almost like berniesanders election http...
2      pupiczech2 im trying gather stuff stay in cat ...
3      annwilburn7 dnc sorry us think ourselves let e...
4      googleexpeuk handyoscar really think needs cal...
                             ...                        
495    justplainanon pattyangel64 koigi3 disagree who...
496    reedgalen corygardner unavailable comment he’s...
497    leftistbitch666 houndsofeah thomasisonline gre...
498    reptdemocracy slovakia’s recent election igor ...
499    realdonaldtrump well learned anything 2016 ele...
Length: 500, dtype: object


In [50]:
#P(A|B) = Probabilidade de relevancia da frase dado suas palavras
#P(B|A) = Probabilidade das palavras da frase acontecerem dado suas relevancias
#P(A)   = Probabilidade de palavras relevantes
#P(B)   = Probabilidade da palavra acontecer

#Número de frases relevantes
r = dados['Relevancia'] == 1
relevantes = dados.loc[r, 'Relevancia']

#Tabela relativa de relevantes
tabela_relevantes = dados.loc[r, 'Treinamento']
rel = ''
for phrase in tabela_relevantes:
    for word in phrase.split():
        rel += ' ' + word
        
rel = clean_words(rel, punct = True, extra_spaces = True)
tabela_relativa_relevantes = pd.Series(rel)
tabela_relativa_relevantes = tabela_relativa_relevantes.value_counts(True)

#Probabilidade de frases relevantes
probR = len(relevantes) / len(tweets)

#Número de frases não relevantes
nr = dados['Relevancia'] == 0
naorelevantes = dados.loc[nr, 'Relevancia']

#Tabela relativa de não relevantes
tabela_nrelevantes = dados.loc[nr, 'Treinamento']
nrel = ''
for phrase in tabela_nrelevantes:
    for word in phrase.split():
        nrel += ' ' + word
        
nrel = clean_words(nrel, punct = True, extra_spaces = True)
tabela_relativa_nrelevantes = pd.Series(nrel)
tabela_relativa_nrelevantes = tabela_relativa_nrelevantes.value_counts(True)

#Probabilidade de frases relevantes
probNR = len(naorelevantes) / len(tweets)

In [51]:
#Calculando a probabilidade das frases serem relevantes ou não usando o teorema de Bayes
probFraseDadoR  = 1
probFraseDadoNR = 1
tabela_prob_relevante = []
tabela_prob_nrelevante = []

#Calculando P(B|A) para relevantes
for phrase in tweets:
    for word in phrase.split():
        try:
            probFraseDadoR *= tabela_relativa_relevantes[word]
        except:
            pass
        
    tabela_prob_relevante.append(probFraseDadoR)
    probFraseDadoR = 1

#Calculando P(B|A) para não relevantes
for phrase in tweets:
    for word in phrase.split():
        try:
            probFraseDadoR *= tabela_relativa_nrelevantes[word]
        except:
            pass
        
    tabela_prob_nrelevante.append(probFraseDadoR)
    probFraseDadoR = 1

#Calculando P(A|B)
probRelevanteDadoF = []
for value in tabela_prob_relevante:
    probRelevanteDadoF.append(value * probR)

probNRelevanteDadoF = []
for value in tabela_prob_nrelevante:
    probNRelevanteDadoF.append(value * probNR)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [52]:
tweets = []
for phrase in dados1['Teste']:
    p = clean(phrase, punct = True, extra_spaces = True)
    p = replaceWords(p)
    tweets.append(p)
            
tweets = pd.Series(tweets)

In [53]:
#Calculando a probabilidade das frases serem relevantes ou não usando o teorema de Bayes
probFraseDadoR  = 1
probFraseDadoNR = 1
tabela_prob_relevante = []
tabela_prob_nrelevante = []

#Calculando P(B|A) para relevantes
for phrase in tweets:
    for word in phrase.split():
        try:
            probFraseDadoR *= tabela_relativa_relevantes[word]
        except:
            pass
        
    tabela_prob_relevante.append(probFraseDadoR)
    probFraseDadoR = 1

#Calculando P(B|A) para não relevantes
for phrase in tweets:
    for word in phrase.split():
        try:
            probFraseDadoR *= tabela_relativa_nrelevantes[word]
        except:
            pass
        
    tabela_prob_nrelevante.append(probFraseDadoR)
    probFraseDadoR = 1

#Calculando P(A|B)
probRelevanteDadoF = []
for value in tabela_prob_relevante:
    probRelevanteDadoF.append(value * probR)

probNRelevanteDadoF = []
for value in tabela_prob_nrelevante:
    probNRelevanteDadoF.append(value * probNR)

___
### Concluindo

In [55]:
Teste = []
relevantes = 0
naorelevantes = 0
acertos = 0
for i in range(len(probRelevanteDadoF)):
    if probRelevanteDadoF[i] < probNRelevanteDadoF[i]:
        Teste.append(1)
        relevantes += 1
    else:
        Teste.append(0)
        naorelevantes += 1
    
    if Teste[len(Teste) - 1] == dados1.at[i, 'Relevancia']:
        acertos += 1
        
print("O algoritmo acertou {0} de {1} tweets.".format(acertos, len(Teste)))

O algoritmo acertou 74 de 124 tweets.


### Conclusão final
O nosso algoritmo acabou acertando cerca de metade dos tweets de teste. Apesar de estarmos usando um método simples de limpeza de texto (retirando algumas palavras irrelevantes, espaços e pontuações) e usando uma classificação binária (relevante ou irrelevante) conseguimos, ao nosso ver, criar um classificador satisfatório com uma complexidade extremamente baixa.
Poderiamos ter olhado de maneira mais cuidadosa a limpeza de texto (Guia muito esclarecedor sobre limpeza de textos para machine learning que usamos para pesquisa: https://machinelearningmastery.com/clean-text-machine-learning-python/), a maneira como classificamos nossos tweets, uma base de dados maior que nos ajudaria a refinar o classificador para vários casos distintos (Usando a API do twitter ou outras redes sociais como Facebook ou Instagram)


___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**